# Spatial Correlation with Permutation Analysis
### Authors:  Mae Morton-Dutton, Christopher Lin, Stephan Palm  
#### Questions Contact: mmorton-dutton@bwh.harvard.edu  
##### Last Updated: January 30, 2024

Use this notebook to find the correlation between 2 datasets using a permutation analysis. 

NOTES:  

- This notebook performs Shan Siddiqi's Spatial Correlation Permutation analysis. Visit [Shan's Paper](https://www.nature.com/articles/s41562-021-01161-1) using this method.  
- Permutation tests can only be used when you have outcome variables for at least one of the datasets , this notebook will not work for only 2 group level maps
    - The outcome variable will be shuffled within each dataset for each permutation
- You will need a csv file that provides the paths to your fcMaps, usually created from the [Preprocessing](https://github.com/nimlab/templates/blob/master/py3_notebooks/1_Preprocessing_LesionQA_fcSurfBcbLqtGen_nimtrack.ipynb) notebook.
- This notebook will output a ground truth spatial correlation between the 2 datasets, ground truth maps, one-sided and two-sided p-values, and a density graph to visualize the permutation test.

In [ ]:
## Packages and environmental settings:

import os
import time
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from nimlab import datasets as nimds
from nimlab import functions as nimfs
from nilearn import image, maskers, masking
from nilearn.maskers import NiftiMasker
from scipy.spatial.distance import cdist

## 1. What type of analysis do you want? and Where is your data?  

### There are **two** types of analysis this notebook can run:
***
### **Option One**: Permute both datasets  
- This is used when you have access to functional maps and outcome for both datasets.

**Instructions**:
- Set `dataset1` and `dataset2` to CSV file paths created from the `Preprocessing` or `xnat_grabber` notebooks that points to your Lesions and fcMaps
- Set `voi1` and `voi2` to the column names containing your primary **Variable of Interest/Outcome Measure** in your CSV files for THIS notebook
- Set `map_type` to the type of functional map you want to use (column name in your CSV file)

For example:
```
dataset1 = 'myDataset1.csv'
voi1 = 'variable1'

dataset2 = 'myDataset2.csv'
voi2 = 'variable2'

map_type = 'avgRFz'
```
***
### **Option Two**: Permute only one dataset
- This is often used when you are testing a dataset against a functional map that is published in literature.
- For this you need functional maps and outcome for one dataset and a group level map for the other dataset

**Instructions**:
- Set `dataset1` to a CSV file path created from the `Preprocessing` or `xnat_grabber` notebooks that points to your Lesions and fcMaps
- Set `voi1` to the column names containing your primary **Variable of Interest/Outcome Measure** in your CSV files for THIS notebook
- Set `dataset2` to the path to the `.nii` file containing the dataset-level functional map for your second dataset
- Leave `voi2` blank
- Set `map_type` to the type of functional map you want to use (column name in your CSV file)

For example:
```
dataset1 = 'myDataset1.csv'
voi1 = 'variable1'

dataset2 = 'myMap.nii.gz'
voi2 = '' # Leave blank if using Option Two

map_type = 'avgRFz'
```
***

In [ ]:
dataset1 = ''
voi1 = ''

dataset2 = '' 
voi2 = '' # Leave blank if using Option Two

# Specify which files are to be used as input (per column name in your input CSVs):
# This is usually 'avgRFz' but the other files could be used as well
# ('avgR', 'avgRFz', or 't')
map_type = 'avgRFz'


# Checking if voi is present
if voi1:
    df1 = pd.read_csv(dataset1)
    if voi1 not in df1:
        raise ValueError(f"{voi1} not in {dataset1}")
    else:
        print("In group 1, I found", len(df1), "participants in", len(df1.dataset.unique()),"different datasets:",df1.dataset.unique())
        print("")
        display(df1)
if voi2:
    df2 = pd.read_csv(dataset2)
    if voi2 not in df2:
        raise ValueError(f"{voi2} not in {dataset2}")
    else:
        print("In group 2, I found", len(df2), "participants in", len(df2.dataset.unique()),"different datasets:",df2.dataset.unique())
        print("")
        display(df2)
else:
    dataset2_name = dataset2.split("/")[-1]
    print(f"\nLoaded {dataset2_name} for dataset2.")

## 2. Load Mask  
### There are **three** options for masking your datasets: 

#### **Option 1 (DEFAULT)** : The default brain mask is `MNI152_T1_2mm_brain_mask_dil` but you can list the name any other masks in [nimds.](https://github.com/nimlab/software_env/blob/e60d97a18a5d5710404d479cfbe6c60270143e20/python_modules/nimlab/nimlab/datasets.py#L10C5-L28C43) 
- Use when individual functional maps in your datasets are created with the same mask
- Insert name of nimds mask (i.e. `mask = 'MNI152_T1_2mm_brain_mask_dil'`)
#### **Option 2**: You can list a path to a custom binary mask file (Nifti). 
- Use when you want to analyze a specific part of the brain instead of whole brain maps
- Insert path to custom mask (i.e. `mask = '/PHShome/mm1360/hippocampusMask.nii.gz'`)
#### **Option 3**: You can mask your maps to the intersection of both datasets (smallest common mask among both dataset1 and dataset2)
- Use when the mask in dataset1 varies slightly from the mask in dataset2
- Leave `mask = ''` blank

In [ ]:
# Load Mask
mask = "MNI152_T1_2mm_brain_mask_dil"
# mask = "" # Use this if you want to use the strict intersection of all your maps

masker = nimfs.loadMask(mask, dataset1, dataset2, map_type)

## 3. Calculate Ground Truth Spatial Correlation  

NOTES:
- You do not need to modify this cell
- This cell will preprocess your data by:
    1) mask your data by the mask specified above 
    2) check for Nan values and incompatible masks
- This cell will also calculate ground truth pearson r maps for each dataset which you can save in the following cell
- Lastly this cell will output the ground truth spatial correlation (Pearson R)

In [ ]:
# Load Images, Get rid of rows with NAN, and reshape VOI
dataset1_img = masker.transform(df1[map_type]).T
print("Checking for NaN values in Dataset 1 ...")
nimfs.checkNan(dataset1_img)
nimfs.checkVar(dataset1_img)
dataset1_voi = np.reshape(np.array(df1[voi1]), (1, -1))
# Calculating ground truth map
dataset1_truthMap = nimfs.genCorrMap(dataset1_img, dataset1_voi).T

if voi2:
    dataset2_img = masker.transform(df2[map_type]).T
    print("Checking for Nan values in Dataset 2 ...")
    nimfs.checkNan(dataset2_img)
    nimfs.checkVar(dataset2_img)
    dataset2_voi = np.reshape(np.array(df2[voi2]), (1, -1))
    dataset2_truthMap = nimfs.genCorrMap(dataset2_img, dataset2_voi).T
    groundTruth = np.corrcoef(
        np.arctanh(dataset1_truthMap), np.arctanh(dataset2_truthMap), rowvar=False
    )[0][1]
else:
    dataset2_truthMap = masker.transform(image.load_img(dataset2)).T
    print("Checking for Nan values in functional map 2 ...")
    nimfs.checkNan(dataset2_truthMap)
    nimfs.checkVar(dataset2_truthMap)
    # Calculating ground truth map
    groundTruth = np.corrcoef(
        np.arctanh(dataset1_truthMap), dataset2_truthMap, rowvar=False
    )[0][1]
print("")
print(f"Ground Truth Spatial Pearson Correlation: {groundTruth}.")

### OPTIONAL: Save Correlation Maps  

NOTES:
- Uncomment and run the cell below if you would like to save the truth maps

In [ ]:
# # Saving truth maps to directory

# # Specify output directory:
# # outdir = './output_folder'
# outdir = ""
# if not os.path.exists(outdir):
#     os.makedirs(outdir)

# # Unmask Data
# dataset1_map = masker.inverse_transform(dataset1_truthMap)
# outname1 = os.path.join(outdir, "dataset1_truthMap.nii.gz")
# dataset1_map.to_filename(outname1)
# print(f"Saved dataset1 truth map to {outname1}")

# dataset2_map = masker.inverse_transform(
#     np.reshape(dataset2_truthMap, (1, -1))
# )
# outname2 = os.path.join(outdir, "dataset2_truthMap.nii.gz")
# dataset2_map.to_filename(outname2)
# print(f"Saved dataset2 to {outname2}")

## 4. Permutation Testing

NOTES:
- This can take awhile depending on the number of subjects, make sure you are running this in a 'protected' session, i.e., jupyter was run in a `tmux` session or an onDemand session.
- Default are `5000` permutations, however `10000` is prefered if time allows.

In [ ]:
# Number Permutations
n_permutations = 5000

start = time.time()

if voi2:
    results = nimfs.defaultPerm(
        n_permutations,
        dataset1_img,
        dataset1_voi,
        dataset2_img,
        dataset2_voi)

else:
    results = nimfs.defaultPerm(
        n_permutations,
        dataset1_img,
        dataset1_voi,
        dataset2_truthMap,
        None)

end = time.time()
print(f"Total Running Time: {end-start}")

## 5. Display Results  

NOTES:
- The one-sided p-value is the most useful for most cases.
    - This will tell you the probability of observing a correlation higher than your ground truth spatial correlation
- The density plot below visualizes the one-sided p-value where the red line is the ground truth spatial correlation

In [ ]:
# 1 tailed positive p-value
greaterThan = 0
for i in results:
    if i > groundTruth:
        greaterThan += 1
print(f"One-Tailed P-Value: {greaterThan/n_permutations}")

# 2 tailed p-value
greaterThan = 0
for i in results:
    if np.abs(i) > np.abs(groundTruth):
        greaterThan += 1
print(f"Two-Tailed P-Value: {greaterThan/n_permutations}")

In [ ]:
# Display Permutation Results
ax = sns.kdeplot(results,fill = True, color = "blue", alpha = 0.1)
ax.axvline(x=groundTruth, color = "red")